# VaR and Expected Shortfall

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import scipy.optimize as opt
from yahoo_fin import stock_info
import pandas as pd
import sys

**Objective:**

To calculate the Value at Risk and Expected Shortfall for a portfolio using the following methods:
1. Historical Simulation
2. Model Building Approach

**1. Historical Simulation:**

In [3]:
class Historical_Simulation():
    def __init__(self,stock_tickers,weights,start_date,end_date):
        self.tickers = stock_tickers
        self.weights = weights
        self.prices = self.get_prices(start_date,end_date)
        self.returns = self.get_returns(1)
    
    def get_prices(self,start_date,end_date):
        s = pd.DataFrame()
        for i in self.tickers:
            b = stock_info.get_data(i,start_date,end_date)['adjclose']
            s[i] =b
        return s.dropna()
    
    def get_returns(self,n):
        returns = pd.DataFrame(data = self.prices.values[n:,]/self.prices.values[:-n,]-1,
                               index = self.prices[n:].index,
                               columns = self.prices.columns)
        
        returns['Portfolio'] = returns.apply(lambda x: np.dot(x,self.weights),axis = 1)
        return returns
    
    
    def calculate_VaR(self,n,confidence):
        r = self.get_returns(n)['Portfolio']
        return np.percentile(r,q = (1-confidence)*100)
    
    def calculate_CVar(self,n,confidence):
        r = self.get_returns(n)['Portfolio']
        VaR = self.calculate_VaR(n,confidence)
        return np.mean(r[r<=VaR])
    
    def change_weights(self,weights):
        self.weights = weights
        self.returns['Portfolio'] = self.returns.drop('Portfolio',axis=1).apply(lambda x: np.dot(x,self.weights),axis = 1)
        

In [179]:
h = Historical_Simulation(['DLF.NS','NTPC.NS','HDFC.NS'],[1,1,1],'2017-10-01','2019-10-01')

In [5]:
w = np.eye(3)
table = pd.DataFrame(np.zeros((3,2)),columns = ['VaR','CVaR'])
for i in range(w.shape[0]):
    h.change_weights(w[i,:])
    table.iloc[i,0] = h.calculate_VaR(1,0.99)
    table.iloc[i,1] = h.calculate_CVar(1,0.99)
table.index = ['DLF.NS','NTPC.NS','HDFC.NS']
print('1-Day, 99% VaR and CVaR for each Stock')
print(np.round(table,2))

1-Day, 99% VaR and CVaR for each Stock
          VaR  CVaR
DLF.NS  -0.07 -0.11
NTPC.NS -0.04 -0.04
HDFC.NS -0.04 -0.05


Above table shows the 1-day, 99% VaR and Expected Shortfall for each stock in proportion to the amount invested.

For example, 1-Day,99% VaR of DLF is 7%. This means that we can be 99% sure that the loss over one day will not exceed 7%.

Similarly, the 1-Day 99% CVaR or Expected Shortfall for DLF is 11%. This means that if the loss on the stock exceeds the 99% VaR, we expect the loss to be an average of 11%.

The reults for an equally weighted portfolio of the three stocks is shown below:

In [185]:
h.change_weights([1,1,1])
print('Equally weighted Portfolio:')
print('1-Day 99% VaR  :',np.round(h.calculate_VaR(1,0.99),3))
print('1-Day 99% CVaR :',np.round(h.calculate_CVar(1,0.99),3))

Equally weighted Portfolio:
1-Day 99% VaR  : -0.106
1-Day 99% CVaR : -0.147


It can be seen the VaR of the portfolio is less than the sum of the VaRs of the individual stocks. This is because of the less than perfect correlation between the stocks. Checking the correlation matrix:

In [182]:
np.corrcoef(h.returns.drop('Portfolio',axis=1),rowvar=False)

array([[1.        , 0.2143877 , 0.29848575],
       [0.2143877 , 1.        , 0.16803448],
       [0.29848575, 0.16803448, 1.        ]])

**2. Model Building Approach**

Model Building Approach assumes a parametric distribution for the returns. The covariance matrix for the returns can be found using different methods. Here we will use the following approaches to calculate volatility and correlation.
1. Based on historical data
2. EWMA Method

In [155]:
class covariance_matrix_EWMA():
    def __init__(self,returns):
        self.returns = returns
        self.l_opt = self.l_optimum()
    
    def cv_update(self,x,y,l=0.94):
        cv = []
        cv.append(x[0]*y[0])
        for i in range(1,len(x)):
            cv.append(l*cv[i-1]+(1-l)*x[i-1]*y[i-1])
        return cv
    
    def cov_matrix(self,l=0.94):
        n = len(self.returns.columns)
        cv_matrix = np.empty(shape=(n,n))
        for i in range(n):
            for j in range(n):
                cv_matrix[i,j] = (self.cv_update(x=self.returns.iloc[:,i].values,
                                     y =self.returns.iloc[:,j].values,
                                     l =l)[-1])
        return cv_matrix
    
    def likelihood_fn(self,cv_matrix):
        if np.ndim(cv_matrix) >1:
            cv_det = np.linalg.det(cv_matrix)
            cv_inv = np.linalg.inv(cv_matrix)
        else:
            cv_det =cv_matrix
            cv_inv = cv_matrix
        f = lambda x: -np.matmul(np.matmul(np.transpose(x),cv_inv),x)-np.log(cv_det)
        logL = self.returns.apply(f,axis=1,raw=True)
        return np.sum(logL)*-1
    
    def obj_fn(self,l_var):
        c = self.cov_matrix(l_var)
        return self.likelihood_fn(c)
        
    def l_optimum(self):
        return opt.minimize_scalar(self.obj_fn,bounds=(0.0001,1),method ='Bounded').x
    
    def get_EWMA_covmatrix(self):
        return self.cov_matrix(self.l_opt)
                

In [156]:
g.get_covmatrix()

array([[7.09156302e-04, 8.57877747e-05, 1.39230412e-04],
       [8.57877747e-05, 2.04510607e-04, 2.87050337e-05],
       [1.39230412e-04, 2.87050337e-05, 2.20007969e-04]])

In [205]:
class model_approach():
    def __init__(self,stock_tickers,amt,start_date,end_date,EWMA=False):
        self.tickers = stock_tickers
        self.EWMA_method = EWMA
        self.amt = amt
        self.weights = amt/np.sum(amt)
        self.prices = self.get_prices(start_date,end_date)
        self.returns = self.get_returns(1)
        self.cv_matrix = self.cv_matrix()
        
    def get_prices(self,start_date,end_date):
        s = pd.DataFrame()
        for i in self.tickers:
            b = stock_info.get_data(i,start_date,end_date)['adjclose']
            s[i] =b
        return s.dropna()
    
    def get_returns(self,n):
        returns = pd.DataFrame(data = self.prices.values[n:,]/self.prices.values[:-n,]-1,
                               index = self.prices[n:].index,
                               columns = self.prices.columns)
        
        return returns
    
    def cv_matrix(self):
        if self.EWMA_method == True:
            c = covariance_matrix_EWMA(self.returns).get_EWMA_covmatrix()
        else:
            c = np.cov(self.returns,rowvar=False)
        return c
    
    def portfolio_variance(self):
        return np.matmul(np.matmul(np.transpose(self.weights),
                         self.cv_matrix),self.weights)
    
    def calculate_VaR(self,n,confidence):
        z = st.norm.ppf(1-confidence)        
        v = self.portfolio_variance()
        return z*(v*n)**0.5*np.sum(self.amt)
    
    def calculate_CVaR(self,n,confidence):
        z = st.norm.ppf(1-confidence)
        Y = self.calculate_VaR(n,confidence)
        v = self.portfolio_variance()
        return -((v*n/(2*np.pi))**0.5)*np.exp(-z**2/2)/(1-confidence)*np.sum(self.amt)
    
    def change_weights(self,amt):
        self.amt = amt
        self.weights = amt/np.sum(amt)
 

In [210]:
m =model_approach(['DLF.NS','NTPC.NS','HDFC.NS'],[1,1,1],'2017-10-01','2019-10-01',EWMA=False)

In [216]:
w = np.eye(3)
table = pd.DataFrame(np.zeros((3,2)),columns = ['VaR','CVaR'])
for i in range(w.shape[0]):
    m.change_weights(w[i,:])
    table.iloc[i,0] = m.calculate_VaR(1,0.99)
    table.iloc[i,1] = m.calculate_CVaR(1,0.99)
table.index = ['DLF.NS','NTPC.NS','HDFC.NS']
print('1-Day, 99% VaR and CVaR for each Stock')
print(np.round(table,2))

1-Day, 99% VaR and CVaR for each Stock
          VaR  CVaR
DLF.NS  -0.06 -0.07
NTPC.NS -0.03 -0.04
HDFC.NS -0.03 -0.04


In [217]:
m.change_weights([1,1,1])
print('Equally weighted Portfolio:')
print('1-Day 99% VaR  :',np.round(m.calculate_VaR(1,0.99),3))
print('1-Day 99% CVaR :',np.round(m.calculate_CVaR(1,0.99),3))

Equally weighted Portfolio:
1-Day 99% VaR  : -0.094
1-Day 99% CVaR : -0.107
